In [2]:
#importing necessary libraries that will be used throughout the project
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

In [3]:
#Load the Subset Dataset
file_path = "cleaned_dataset.csv"  
df = pd.read_csv(file_path)

In [4]:
# Encoding Categorical Features
label_encoder = LabelEncoder()
if 'Label' in df.columns:
    df['Label'] = label_encoder.fit_transform(df['Label'])